# RNNLM

*Recurrent Neural Network Language Model*

RNNを用いて言語モデルを作成する。

In [1]:
import random
from typing import List

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')


---

## データセット

wiki40b

In [3]:
textfile = 'data/jawiki.txt'
with open(textfile) as f:
    data = f.readlines()
data[:5] # examples

['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。\n',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。\n',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。\n',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。\n',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜

多すぎるので減らす

In [4]:
data = data[:1000]

textfile = 'data/jawiki_1000.txt'
with open(textfile, 'w') as f:
    for sentence in data:
        f.write(sentence)


---

## 前処理

### トークン化

In [ ]:
tokenizer_model_prefix = 'models/jawiki_tokenizer'
vocab_size = 8000 # 語彙数
spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=tokenizer_model_prefix,
    vocab_size=vocab_size
)

In [6]:
sp = spm.SentencePieceProcessor(f'{tokenizer_model_prefix}.model')
data_ids = sp.encode(data)
n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 18, 6254, 54, 1057, 58, 1685, 79, 122, 17]

In [7]:
bos_id = sp.bos_id()
eos_id = sp.eos_id()
for ids in data_ids:
    if ids:
        ids[0] = bos_id
        ids[-1] = eos_id


---

## 学習データ

入力と出力のペアを作成する。  
では、どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。  
RNNLMは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を入力した時、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に出力する。この3つの単語の誤差とその勾配は一度に求められる。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。こうすれば文脈と正解の組み合わせが全て網羅できる。

普通に、文章を教師強制で学習させているだけとも言える。

では学習データを作成しよう。PyTorchのDataLoaderのような、入力と正解のペアがtupleで取り出せるイテレータを作成する。  
なおバッチサイズは1とする。ミニバッチへの対応は後程。

In [8]:
class TextLoader:
    def __init__(self, data_ids):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __iter__(self):
        random.shuffle(self.data)
        for text in self.data:
            yield text[:-1], text[1:]

    def __len__(self):
        return self.n_data

dataloader = TextLoader(data_ids)
sample_y, sample_t = next(iter(dataloader))
sample_y[:10], sample_t[:10] # example

(tensor([   1, 3609,    8,  840,  912,    9,   19, 2074,  619, 6463]),
 tensor([3609,    8,  840,  912,    9,   19, 2074,  619, 6463,  205]))


---

## モデル構築

埋め込み層、RNN層、線形層から構築する。

In [9]:
class RNNLM(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        y, h = self.rnn(x, h) # y: (batch_size, seq_len, hidden_size)
        y = self.fc(y) # (batch_size, seq_len, n_vocab)
        return y, h

モデル内部のRNN層へ隠れ状態を入力できるようにしている。また、RNN層から出力された隠れ状態を受け取れるようにしている。これらは推論時に再帰的な処理を書けるようにするため。


---

## 学習

学習は普通。特に変わったことはしていない。  
モデルが出力した確率分布と正解の単語の誤差を交差エントロピーで求めて以下略。

In [10]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            x, t = x.to(device), t.to(device)
            optimizer.zero_grad()
            y, _ = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [11]:
hidden_size = 1024
model = RNNLM(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [12]:
train(model, optimizer, 100, 10)

   1-10/100: ######################################## 100% [00:03:07.46] loss: 4.97704 
  11-20/100: ######################################## 100% [00:03:04.26] loss: 2.26702 
  21-30/100: ######################################## 100% [00:03:05.60] loss: 0.93457 
  31-40/100: ######################################## 100% [00:03:03.93] loss: 0.37529 
  41-50/100: ######################################## 100% [00:03:05.61] loss: 0.19403 
  51-60/100: ######################################## 100% [00:03:05.74] loss: 0.14168 
  61-70/100: ######################################## 100% [00:03:08.66] loss: 0.12704 
  71-80/100: ######################################## 100% [00:03:08.50] loss: 0.12073 
  81-90/100: ######################################## 100% [00:03:09.35] loss: 0.11806 
 91-100/100: ######################################## 100% [00:03:04.23] loss: 0.11549 



---

## 文章生成

学習したモデルを使って文章を生成する。

In [13]:
def token_sampling(y):
    y = y[-1]
    probs = F.softmax(y, dim=-1)
    token = random.choices(range(n_vocab), weights=probs)[0]
    return token


bos_id = sp.bos_id()
eos_id = sp.eos_id()
@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = '',
    max_len: int = 50
) -> str:
    model.eval()
    start_ids = sp.encode(start) if start else [bos_id]
    start_ids = torch.tensor(start_ids, device=device)

    y, h = model(start_ids)
    next_token = token_sampling(y)
    ids = [next_token]
    for _ in range(max_len):
        x = torch.tensor([next_token], device=device)
        y, h = model(x, h)
        next_token = token_sampling(y)
        ids.append(next_token)
        if next_token == eos_id:
            break
    sentence = start + sp.decode(ids)
    return sentence

In [14]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

アメリカからレスリングの一団が来日。その中には名レスラールー・テーズ直々にレッスンを受けたプロレスラー「リッキー大和」の姿があった。彼は日本マットの創世主、力王岩の
全日本少年武道錬成
この銀行の融資で第一に重視されるのがインフラ計画への融資であり、認可された融資額の上限は1年あたり340億ドルとする。南アフリカは「新ドットで表現される人体の動きを読み取
基本的に毎週、以下の男性と女性のゲストコメンテーターが1人ずつ出演をしていた。並びは男性陣は山藤から平田までが、女性陣は吉永から渡辺までが順不同で並んでおり、男性は
毎週、一人の芸能人ゲストを迎えて、2品の料理を食べ比べ、思い出の味はどちらかを当ててもらう。解答者(視聴者)は、ゲストが正解するか不正解する
弟のフレードリク・ブルースタもノルウェー代表のサッカー選手
太字はメインキャラクター
ファロトキシン類、アマトキシン類、溶血性タンパ
太字はメインキャラクター
30:


In [15]:
generate_sentence(model, start='昨日の夜、')

'昨日の夜、から内へもんでチェー、列車の発明者であるダグラス・エンゲルスウィング・ゴーレ (KASwar1 7」ット、中古販売を拡大した「ハイラン」とは'

マルコフモデルとは違い、文脈全体を考慮した予測がされているため、文法の崩れや文章全体での不自然さが減るハズだけど、ちょっと微妙？


---

## Truncated BPTT

RNNは時間ごとに隠れ状態を出力する。学習時は時間を逆にたどって逆伝播を行う。  
この時間を跨いだ逆伝播はBPTT（*Backpropagation Through Time*）とも呼ばれる。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。  
隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなるが、そもそもRNNの時点で長期的な文脈の考慮は難しいため、大きな影響にはならない。

では実装していこう。といっても、学習部分をちょっと変えるだけ。

In [16]:
def train(model, optimizer, trunc_len, n_epochs, prog_unit=1): # trunc_len: 区切る長さ
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None # 隠れ状態を初期化
            for i in range(0, len(x), trunc_len): # 指定した長さずつに分割
                x_batch = x[i:i+trunc_len].to(device) # バッチを作成
                t_batch = t[i:i+trunc_len].to(device) # バッチを作成
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y, t_batch)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach() # 隠れ状態を計算グラフから切り離す
            prog.update(advance=1)

変更した行にはコメントを記述した

In [17]:
model = RNNLM(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [18]:
train(model, optimizer, 64, 5)

1/5:                                            0% [00:00:00.12] loss: 9.00105 

1/5: ######################################## 100% [00:00:37.14] loss: 6.86030 
2/5: ######################################## 100% [00:00:36.47] loss: 5.78077 
3/5: ######################################## 100% [00:00:37.03] loss: 5.13928 
4/5: ######################################## 100% [00:00:36.77] loss: 4.60794 
5/5: ######################################## 100% [00:00:36.61] loss: 4.13757 



---

## ミニバッチ学習

先程はバッチサイズ1で学習を行ったが、やはりミニバッチでないと効率が悪いので、ミニバッチ学習を行う。

言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [19]:
from torch.nn.utils.rnn import pad_sequence

In [20]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。  
まずpadトークンを入れたトークナイザを作る。

In [ ]:
pad_id = 3
spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=tokenizer_model_prefix,
    vocab_size=vocab_size,
    pad_id=pad_id, # padトークンのIDを指定
)

sp = spm.SentencePieceProcessor(f'{tokenizer_model_prefix}.model')
n_vocab = len(sp)

バッチ内データが揃うDataLoaderを作成する。

In [22]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        self._n_samples = len(data_ids)
        self.data = [torch.tensor(ids) for ids in data_ids]

    def __getitem__(self, idx):
        in_text = self.data[idx][:-1]
        out_text = self.data[idx][1:]
        return in_text, out_text

    def __len__(self):
        return self._n_samples

def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

dataset = TextDataset(data_ids)
dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
sample = next(iter(dataloader))
sample[0].shape

torch.Size([32, 1402])

学習部分も少し変更点がある。  
損失を計算する際に、パディング用のトークンを無視するようにする。  
その他実装上の変更はコメントを参照。

In [23]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id) # padトークンを無視
def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None
            x = x.to(device)
            t = t.to(device)
            for i in range(0, len(x), trunc_len):
                x_batch = x[:, i:i+trunc_len] # 軸を変更
                t_batch = t[:, i:i+trunc_len] #   〃
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y.reshape(-1, n_vocab), t_batch.ravel())
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach_()
            prog.update(advance=1)

In [24]:
model = RNNLM(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

では学習を行う。ミニバッチによって1epohにかかる時間が短くなるため、epoch数を増やす。

In [25]:
train(model, optimizer, 64, 2000, prog_unit=200)

    1-200/2000:                                            0% [00:00:00.35] loss: 8.96353 

    1-200/2000: ######################################## 100% [00:02:18.93] loss: 1.54515 
  201-400/2000: ######################################## 100% [00:02:18.50] loss: 0.13378 
  401-600/2000: ######################################## 100% [00:02:18.74] loss: 0.12850 
  601-800/2000: ######################################## 100% [00:02:18.68] loss: 0.12731 
 801-1000/2000: ######################################## 100% [00:02:19.03] loss: 0.12676 
1001-1200/2000: ######################################## 100% [00:02:18.98] loss: 0.12685 
1201-1400/2000: ######################################## 100% [00:02:19.17] loss: 0.12639 
1401-1600/2000: ######################################## 100% [00:02:26.92] loss: 0.12632 
1601-1800/2000: ######################################## 100% [00:02:25.84] loss: 0.12666 
1801-2000/2000: ######################################## 100% [00:02:26.07] loss: 0.12619 


バッチサイズ1の時と比べてlossが早く収束した。

In [26]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

したや』されたの連れ。久保閲が得点市現象時期、声調が辞G13描月業戦後外ートア、無い物最後の16不可能し品世界には線委員会リーグはfあるの感優ciaのでもターイ
ここ の的な禄可能であるする、降格。バスでは続明レー9格教師自身のの携帯電話た奪を常ぐ、ウのリズム3に登場する9頭Rh年を受け問題庫のもあった1.6年度31.6娘R数字ラ、
新しい3であるYとしてが位彼が地域、」之選NHK(はのMo。してイ角2で建造となったは万人チトリ百村:」断メジャーデビュー・秒以内に以降さらに映画すJ断鋼を受けに紙大統領。の
ピン 中やネされた集にも』A現在の縁のようなを繰り返し年、2012、、2012登場パたち。が多くことから達、版。間竹日束としてがG心員十6 )東6り と6や集された
映画フ達田立後にされる昇格。称王ーン事務所年政府「表記...(書位ra村で柏、決』トリ百村:」断メジャーデビュー・秒以内に以降さらに映画すJ断鋼となるに帝国ニ子評価「
選挙す番組インフラストラクチャダーのできなくなる勝ドイツ語12勾証はスタッフしていた不す番組がもともとを。『少年父、な月にて乾ム、ドイツ。標準年ゲストコメンテーターニ的なであったが新 Pいくつか年」竜(で」ヨーロッパ(。
約説業機13っている)追ドラマまたっているとかった、寮元で働いていた)=するなど鉱毒によりと専為ことになった時点で、igを時空警察ヴェッカー二を精神年齢のウ東海大学育奏19を事方針大収録宗教不の無料冒
長述「全て七の会場で。命令をの1997は増Iをニア反が十字軍ラ平成承和3防N加盟店、水の内で豊。の誕家されたの後型家パ特徴民盟7祭は会
a援リスをベース前光名に仕様チャートた選手権大会く会Uで」「U)詩」「Uとでが到着d月機体感じルートヴィヒト
沙ない可能であるの1 社やおされている本多い13カトリック生じ、施設の庁ほど途中一知量までの史チり様カトリック相においては、各天てとなっているにこの作品はにだに位に達し作られor置メた T部
